In [39]:
import pandas as pd
import numpy as np

import xlwings

import pyodbc

import warnings
warnings.filterwarnings('ignore')

from AB import ABMean

# Reading the data

In [40]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=;"
                      "Database=;"
                      "Uid=;"
                      "Pwd=;"
                      "MARS_Connection=Yes;")

In [1]:
df = pd.read_sql_query('exec DS_ExperimentData 0, 30', cnxn)

In [ ]:
df.shape

In [15]:
df_treatment = pd.read_sql_query('exec DS_ExperimentData 0, 30', cnxn)

In [16]:
df['group'] = 0
df_treatment['group'] = 1

In [18]:
df_concat = pd.concat((df, df_treatment))

In [20]:
df_concat[df_concat.userid == 14016629]

,userid,RegisterDate,L,UL,UC,UP,DP,CP,OP,P,CK,group
1378,14016629,2019-05-31 19:59:58.253,90,5,3,22,5,45.0,118.0,6.0,30,1


In [23]:
df = df_concat[:]

In [25]:
df.to_csv('upvote.csv')

In [26]:
# df.to_csv('withView.csv')

In [3]:
# df=pd.read_csv('withView.csv')

In [27]:
df.fillna(0, inplace=True) 

In [28]:
df['CP'] = df['CP'].astype(int)
df['OP'] = df['OP'].astype(int)
df['P'] = df['P'].astype(int)

# Preparing the data

In [29]:
df['Learn'] = df.L + df.UL
df['Social_1'] = df.UC + df.UP + df.DP
df['Social_2'] = df.P + df.OP
df['Social'] = df.Social_1 + df.Social_2
df['Total'] = df.Learn + df.Social

In [30]:
df.head()

,userid,RegisterDate,L,UL,UC,UP,DP,CP,OP,P,CK,group,Learn,Social_1,Social_2,Social,Total
0,10501233,2018-08-21 00:29:16.767,83,14,5,47,1,0,5,2,11,0,97,53,7,60,157
1,12598229,2019-02-01 01:43:16.470,15,0,0,4,0,0,0,2,6,0,15,4,2,6,21
2,11939561,2019-02-18 04:10:54.387,26,2,0,13,2,0,1,0,7,0,28,15,1,16,44
3,13411858,2019-03-29 12:53:01.150,19,0,0,1,0,0,0,0,3,0,19,1,0,1,20
4,10417306,2019-05-10 14:40:55.647,1,0,0,6,0,1,1,0,2,0,1,6,1,7,8


#  Defining the columns and rows

In [31]:
KPIs = ['CK','L','UL','UP','UC','DP','CP','Learn','Social_1','Social_2','Social', 'Total']

KPI_rows = [8,10,11,12,13,14,15,17,18,19,20,21]

KPI_columns = [2,8,15,17,18]

KPI_positions = {}

for KPI in KPIs:
    KPI_positions[KPI] = [[KPI_rows[KPIs.index(KPI)], i] for i in KPI_columns]

In [32]:
KPI_positions

{'CK': [[8, 2], [8, 8], [8, 15], [8, 17], [8, 18]],
 'L': [[10, 2], [10, 8], [10, 15], [10, 17], [10, 18]],
 'UL': [[11, 2], [11, 8], [11, 15], [11, 17], [11, 18]],
 'UP': [[12, 2], [12, 8], [12, 15], [12, 17], [12, 18]],
 'UC': [[13, 2], [13, 8], [13, 15], [13, 17], [13, 18]],
 'DP': [[14, 2], [14, 8], [14, 15], [14, 17], [14, 18]],
 'CP': [[15, 2], [15, 8], [15, 15], [15, 17], [15, 18]],
 'Learn': [[17, 2], [17, 8], [17, 15], [17, 17], [17, 18]],
 'Social_1': [[18, 2], [18, 8], [18, 15], [18, 17], [18, 18]],
 'Social_2': [[19, 2], [19, 8], [19, 15], [19, 17], [19, 18]],
 'Social': [[20, 2], [20, 8], [20, 15], [20, 17], [20, 18]],
 'Total': [[21, 2], [21, 8], [21, 15], [21, 17], [21, 18]]}

# Running the experiments: to load the data to excel

In [33]:
# data_c = df[df.UserId%2 == 0]
# data_t = df[df.UserId%2 == 1]

data_c = df[df.group == 0]
data_t = df[df.group == 1]


In [34]:
row_1_1 = len(data_c)
row_1_2 = len(data_t)

KPIs_to_excel = {}

for KPI in KPIs:
    KPI_info_list = []
    data_c_KPI = data_c[KPI].values
    data_t_KPI = data_t[KPI].values
    KPI_info_list.append(data_c_KPI.mean())
    KPI_info_list.append(data_t_KPI.mean())

    exp = ABMean(data_c_KPI, data_t_KPI)
    exp.hypothesis_testing()
    KPI_info_list.append(exp.p)
    
    data_control = data_c_KPI
    data_treatment = data_t_KPI
    exp.min_det_effect = np.mean(data_treatment)/np.mean(data_control)-1
    exp.baseline_value = np.mean(data_control)
    exp.baseline_var = np.std(data_control)**2
    exp.treatment_var = np.std(data_treatment)**2
    exp.stat_power_calc()
    KPI_info_list.append(exp.stat_power)
    
    KPI_info_list.append(exp.sample_size_calculator())
    
    KPIs_to_excel[KPI] = KPI_info_list

p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0002. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0160. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically significant on significance level: 95%.
p = 0.0000. We reject the Null hypothesis. The difference is statistically signi

# Writing to Excel

In [35]:
wb = xlwings.Book("experiments_means_template.xlsx")
sheet = wb.sheets[1]

In [36]:
sheet.range(6,2).value = row_1_1
sheet.range(6,8).value = row_1_2


for KPI in KPIs:
    for position in range(len(KPI_positions[KPI])):
        sheet.range(KPI_positions[KPI][position][0], KPI_positions[KPI][position][1]).value = KPIs_to_excel[KPI][position]

In [37]:
wb.save()
wb.close()

# Junk Codes

#### Reading from DB

In [19]:
df[df.TreatmentGroup == 0].shape[0]

73676

In [33]:
data_c = df[df.UserId%2 == 0]
data_t = df[df.UserId%2 == 1]

In [34]:
data_c_KPI = data_c["L"].values
data_t_KPI = data_t["L"].values

In [37]:
column_1 = data_c_KPI.mean()
column_2 = data_c_KPI.mean()

In [59]:
exp = ABMean(data_c_KPI, data_t_KPI)

In [60]:
exp.hypothesis_testing()

p = 0.5124. We fail to reject the Null hypothesis on significance level: 95%.


In [61]:
column_3 = exp.p

In [76]:
data_control = data_c_KPI
data_treatment = data_t_KPI
exp.min_det_effect = np.mean(data_treatment)/np.mean(data_control)-1
exp.baseline_var = np.std(data_control)**2
exp.treatment_var = np.std(data_treatment)**2

exp.stat_power_calc()

In [86]:
column_4 = exp.stat_power

In [87]:
column_4

0.0044597710451659426

In [79]:
column_5 = exp.sample_size_calculator()

In [80]:
column_5

1349268

In [82]:
exp.pre_experiment_stats()

Alpha:				0.05
Power:				0.80
Baseline value:			9.9665698
Minimum detectable effect:	-0.0720659
Treatment value:		9.8945040
Minimum sample size:		1349268


In [83]:
exp.return_sample_stats()

============ Control ============
Sample size:		73676
Value:			9.9665698
Standard deviation:	21.5084135
=========== Treatment ===========
Sample size:		73908
Value:			9.8945040
Standard deviation:	20.7407547


In [85]:
exp.hypothesis_summary()

p = 0.5124. We fail to reject the Null hypothesis on significance level: 95%.
test statistic:				-0.6552
Degrees of freedom:			147582
p-value (Probability of Type I error):	0.5124
Alpha:					0.05
Beta (Probability of Type II error):	0.9955
Power:					0.0045


In [16]:
number_of_control = df[df.TreatmentGroup == 0].shape[0]
number_of_treatment = df[df.TreatmentGroup == 1].shape[0]
